In [72]:
import pandas as pd
import string
import re
import nltk
import spacy
import json

In [157]:
# Read the JSON file
results_json = pd.read_json("bq-results-20240124-055833-1706076079048.json", lines=True)

# Read the first CSV file
ct_media = pd.read_csv("cleantech_media_dataset_v2_2024-02-23.csv")

# Read the second CSV file
ct_evaluation = pd.read_csv("cleantech_rag_evaluation_data_2024-02-23.csv")


In [158]:
# Display the first few rows of the DataFrame
print("First few rows:")
print(results_json.head())


First few rows:
  publication_number application_number country_code  \
0   US-2022239235-A1  US-202217717397-A           US   
1   US-2022239251-A1  US-202217580956-A           US   
2      EP-4033090-A1      EP-21152924-A           EP   
3      EP-4033090-A1      EP-21152924-A           EP   
4     US-11396827-B2  US-202117606042-A           US   

                                     title_localized  \
0  [{'text': 'Adaptable DC-AC Inverter Drive Syst...   
1  [{'text': 'System for providing the energy fro...   
2  [{'text': 'Verfahren zum steuern einer windene...   
3  [{'text': 'Verfahren zum steuern einer windene...   
4  [{'text': 'Control method for optimizing solar...   

                                  abstract_localized  publication_date  \
0  [{'text': 'Disclosed is an adaptable DC-AC inv...          20220728   
1  [{'text': 'In accordance with an example embod...          20220728   
2  [{'text': 'Verfahren zum Steuern einer Windene...          20220727   
3  [{'text': '

In [159]:
# Display basic information about the DataFrame
print("\nData information:")
print(results_json.info())


Data information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   publication_number  30000 non-null  object
 1   application_number  30000 non-null  object
 2   country_code        30000 non-null  object
 3   title_localized     30000 non-null  object
 4   abstract_localized  30000 non-null  object
 5   publication_date    30000 non-null  int64 
 6   inventor            30000 non-null  object
 7   cpc                 30000 non-null  object
dtypes: int64(1), object(7)
memory usage: 1.8+ MB
None


In [160]:
# Display summary statistics for numerical columns
print("\nSummary statistics:")
print(results_json.describe())


Summary statistics:
       publication_date
count      3.000000e+04
mean       2.021843e+07
std        3.783564e+03
min        2.021112e+07
25%        2.022011e+07
50%        2.022032e+07
75%        2.022052e+07
max        2.022073e+07


In [161]:
print("First few rows:")
print(ct_media.head())

First few rows:
   Unnamed: 0                                              title        date  \
0        1280  Qatar to Slash Emissions as LNG Expansion Adva...  2021-01-13   
1        1281               India Launches Its First 700 MW PHWR  2021-01-15   
2        1283              New Chapter for US-China Energy Trade  2021-01-20   
3        1284  Japan: Slow Restarts Cast Doubt on 2030 Energy...  2021-01-22   
4        1285     NYC Pension Funds to Divest Fossil Fuel Shares  2021-01-25   

  author                                            content       domain  \
0    NaN  ["Qatar Petroleum ( QP) is targeting aggressiv...  energyintel   
1    NaN  ["• Nuclear Power Corp. of India Ltd. ( NPCIL)...  energyintel   
2    NaN  ["New US President Joe Biden took office this ...  energyintel   
3    NaN  ["The slow pace of Japanese reactor restarts c...  energyintel   
4    NaN  ["Two of New York City's largest pension funds...  energyintel   

                                              

In [162]:
print("First few rows:")
print(ct_evaluation.head())

First few rows:
   example_id  question_id                                           question  \
0           1            1  What is the innovation behind Leclanché's new ...   
1           2            2       What is the EU’s Green Deal Industrial Plan?   
2           3            2       What is the EU’s Green Deal Industrial Plan?   
3           4            3  What are the four focus areas of the EU's Gree...   
4           5            4  When did the cooperation between GM and Honda ...   

                                      relevant_chunk  \
0  Leclanché said it has developed an environment...   
1  The Green Deal Industrial Plan is a bid by the...   
2  The European counterpart to the US Inflation R...   
3  The new plan is fundamentally focused on four ...   
4  What caught our eye was a new hookup between G...   

                                         article_url  
0  https://www.sgvoice.net/strategy/technology/23...  
1  https://www.sgvoice.net/policy/25396/eu-seeks-.

In [163]:
#Drop duplicates CSV
#ct_media.drop_duplicates(inplace=True)
#ct_evaluation.drop_duplicates(inplace=True)


In [164]:
print(results_json.head())

  publication_number application_number country_code  \
0   US-2022239235-A1  US-202217717397-A           US   
1   US-2022239251-A1  US-202217580956-A           US   
2      EP-4033090-A1      EP-21152924-A           EP   
3      EP-4033090-A1      EP-21152924-A           EP   
4     US-11396827-B2  US-202117606042-A           US   

                                     title_localized  \
0  [{'text': 'Adaptable DC-AC Inverter Drive Syst...   
1  [{'text': 'System for providing the energy fro...   
2  [{'text': 'Verfahren zum steuern einer windene...   
3  [{'text': 'Verfahren zum steuern einer windene...   
4  [{'text': 'Control method for optimizing solar...   

                                  abstract_localized  publication_date  \
0  [{'text': 'Disclosed is an adaptable DC-AC inv...          20220728   
1  [{'text': 'In accordance with an example embod...          20220728   
2  [{'text': 'Verfahren zum Steuern einer Windene...          20220727   
3  [{'text': 'Verfahren zum St

#### Change date format

In [165]:
# Change JSON date format
results_json['publication_date'] = pd.to_datetime(results_json['publication_date'], format='%Y%m%d')


#### Convert "cpc" column 

In [166]:
# Inspect cpc column and handle it, second try #first time it worked and then it did not, so I might have a code that overwrite the column
print("Checking 'cpc' column:")
print(results_json['cpc'].head())

Checking 'cpc' column:
0    [{'code': 'H02M7/5395', 'inventive': True, 'fi...
1    [{'code': 'H02S40/38', 'inventive': True, 'fir...
2    [{'code': 'F03D7/0276', 'inventive': True, 'fi...
3    [{'code': 'F03D7/0276', 'inventive': True, 'fi...
4    [{'code': 'F24S50/00', 'inventive': True, 'fir...
Name: cpc, dtype: object


In [167]:
# Redefine the function for extracting the first CPC entry
def extract_first_cpc(cpc_entry):
    if cpc_entry and isinstance(cpc_entry, list) and len(cpc_entry) > 0:
        return cpc_entry[0]
    return None

# Apply the function to the 'cpc' column
results_json['cpc'] = results_json['cpc'].apply(extract_first_cpc)

# Verify that the function worked as expected
print("\nVerifying 'cpc' after applying extract_first_cpc function:")
print(results_json['cpc'].head())


Verifying 'cpc' after applying extract_first_cpc function:
0    {'code': 'H02M7/5395', 'inventive': True, 'fir...
1    {'code': 'H02S40/38', 'inventive': True, 'firs...
2    {'code': 'F03D7/0276', 'inventive': True, 'fir...
3    {'code': 'F03D7/0276', 'inventive': True, 'fir...
4    {'code': 'F24S50/00', 'inventive': True, 'firs...
Name: cpc, dtype: object


In [168]:
# Assuming 'cpc' is now properly formatted, apply lambda functions to create new columns
results_json['cpc_code'] = results_json['cpc'].apply(lambda x: x.get('code') if isinstance(x, dict) and 'code' in x else None)
results_json['cpc_inventive'] = results_json['cpc'].apply(lambda x: x.get('inventive') if isinstance(x, dict) and 'inventive' in x else None)
results_json['cpc_first'] = results_json['cpc'].apply(lambda x: x.get('first') if isinstance(x, dict) and 'first' in x else None)
results_json['cpc_tree'] = results_json['cpc'].apply(lambda x: x.get('tree') if isinstance(x, dict) and 'tree' in x else None)

# Now, let's check the new columns to ensure they contain the expected data
print("\nFinal 'cpc' related columns:")
print(results_json[['cpc_code', 'cpc_inventive', 'cpc_first', 'cpc_tree']].head())


Final 'cpc' related columns:
     cpc_code cpc_inventive cpc_first cpc_tree
0  H02M7/5395          True     False       []
1   H02S40/38          True     False       []
2  F03D7/0276          True      True       []
3  F03D7/0276          True      True       []
4   F24S50/00          True     False       []


In [169]:
# Verify the columns have been dropped by printing the DataFrame's head
print(results_json.head())

  publication_number application_number country_code  \
0   US-2022239235-A1  US-202217717397-A           US   
1   US-2022239251-A1  US-202217580956-A           US   
2      EP-4033090-A1      EP-21152924-A           EP   
3      EP-4033090-A1      EP-21152924-A           EP   
4     US-11396827-B2  US-202117606042-A           US   

                                     title_localized  \
0  [{'text': 'Adaptable DC-AC Inverter Drive Syst...   
1  [{'text': 'System for providing the energy fro...   
2  [{'text': 'Verfahren zum steuern einer windene...   
3  [{'text': 'Verfahren zum steuern einer windene...   
4  [{'text': 'Control method for optimizing solar...   

                                  abstract_localized publication_date  \
0  [{'text': 'Disclosed is an adaptable DC-AC inv...       2022-07-28   
1  [{'text': 'In accordance with an example embod...       2022-07-28   
2  [{'text': 'Verfahren zum Steuern einer Windene...       2022-07-27   
3  [{'text': 'Verfahren zum Steuer

#### Handle Missing in "inventor" and "cpc_tree"

In [170]:
# Check if all entries in 'cpc_tree' are empty lists or None
all_empty_trees = all(x == [] or x is None for x in results_json['cpc_tree'])
print("All entries in 'cpc_tree' are empty lists or None:", all_empty_trees)


All entries in 'cpc_tree' are empty lists or None: True


In [171]:
# Drop the 'cpc_tree' column
results_json.drop(['cpc_tree'], axis=1, inplace=True)

In [172]:
# Drop the 'cpc' column
results_json.drop(['cpc'], axis=1, inplace=True)

In [173]:
# Calculate the percentage of non-empty entries in 'inventor'
non_empty_inventors = results_json['inventor'].apply(bool).mean() * 100
print(f"Percentage of non-empty 'inventor' entries: {non_empty_inventors}%")


Percentage of non-empty 'inventor' entries: 63.64666666666666%


In [174]:
# Standardize the names in the 'inventor' column and handle missing data
results_json['inventor'] = results_json['inventor'].apply(lambda x: [name.strip().title() for name in x] if x else None)

# Create a new column for the count of inventors
results_json['num_inventors'] = results_json['inventor'].apply(lambda x: len(x) if x else 0)

# Create a binary indicator for multiple inventors
results_json['multiple_inventors'] = results_json['num_inventors'].apply(lambda x: 1 if x > 1 else 0)

# Check the results
print(results_json[['inventor', 'num_inventors', 'multiple_inventors']].head())

# Analyze the distribution of the number of inventors
print("Distribution of number of inventors:")
print(results_json['num_inventors'].value_counts())

                                            inventor  num_inventors  \
0                                               None              0   
1                                               None              0   
2  [Schaper, Ulf, Von Aswege, Enno, Gerke Funcke,...              3   
3  [Schaper, Ulf, Von Aswege, Enno, Gerke Funcke,...              3   
4                                               None              0   

   multiple_inventors  
0                   0  
1                   0  
2                   1  
3                   1  
4                   0  
Distribution of number of inventors:
0     10906
1      6771
2      2636
3      2551
4      1741
5      1595
6      1155
7       799
8       543
9       328
10      287
11      157
12      134
13       92
15       86
14       70
16       31
18       27
17       24
20       22
21       18
19       14
25        4
22        3
24        2
38        2
29        2
Name: num_inventors, dtype: int64


#### Process publication and application number columns

In [175]:
# Check for null values in 'country_code', to see if I can safely remove codes integrated in publication adn application column
null_country_codes = results_json['country_code'].isnull().sum()
print(f"Number of null entries in 'country_code': {null_country_codes}")


Number of null entries in 'country_code': 0


In [176]:
import re

# Define a function to remove the prefix and suffix
def remove_prefix_suffix(text):
    # Remove the country code prefix
    text = re.sub(r'^[A-Za-z]+-', '', text)
    # Remove any suffix that starts with a dash after the numeric sequence
    text = re.sub(r'-.+$', '', text)
    return text

# Apply the function to the publication and application number columns
results_json['publication_number'] = results_json['publication_number'].apply(remove_prefix_suffix)
results_json['application_number'] = results_json['application_number'].apply(remove_prefix_suffix)

# Display the DataFrame to confirm the changes
print(results_json[['publication_number', 'application_number']].head())

  publication_number application_number
0         2022239235       202217717397
1         2022239251       202217580956
2            4033090           21152924
3            4033090           21152924
4           11396827       202117606042


### Text cleaning 

In [183]:
import pandas as pd

# Assuming 'results_json' is your DataFrame
# Check for null values in specific columns (adjust column names as per your DataFrame)
print(results_json['abstract_text'].isnull().sum())
print(results_json['title_text'].isnull().sum())


0
0


In [184]:
# Check for empty or whitespace-only strings
empty_abstracts_count = results_json[results_json['abstract_text'].apply(lambda x: x.strip() == '')].shape[0]
empty_titles_count = results_json[results_json['title_text'].apply(lambda x: x.strip() == '')].shape[0]

print(f"Number of effectively empty abstracts: {empty_abstracts_count}")
print(f"Number of effectively empty titles: {empty_titles_count}")


Number of effectively empty abstracts: 11775
Number of effectively empty titles: 13554


In [185]:
# Display some samples of entries that might be causing zero tokens
print("Sample entries from abstract_text:")
print(results_json[results_json['abstract_text'].apply(lambda x: x.strip() == '')]['abstract_text'].head())

print("Sample entries from title_text:")
print(results_json[results_json['title_text'].apply(lambda x: x.strip() == '')]['title_text'].head())


Sample entries from abstract_text:
5     
9     
14    
15    
19    
Name: abstract_text, dtype: object
Sample entries from title_text:
9     
10    
12    
13    
22    
Name: title_text, dtype: object


In [177]:
def extract_text_from_json(row):
    if row and isinstance(row, list):
        try:
            # Extract the 'text' part from the first dictionary in the list
            return row[0]['text']
        except (IndexError, KeyError):
            # Return None or some default value if there's an error in parsing
            return None
    return None

# Applying the updated function to 'abstract_localized' and 'title_localized' to extract the 'text' part
results_json['abstract_text'] = results_json['abstract_localized'].apply(extract_text_from_json)
results_json['title_text'] = results_json['title_localized'].apply(extract_text_from_json)

# Drop the original columns if they are no longer needed
results_json.drop(columns=['abstract_localized', 'title_localized'], inplace=True)

# Inspect the cleaned columns
print(results_json[['abstract_text', 'title_text']].head())


                                       abstract_text  \
0  Disclosed is an adaptable DC-AC inverter syste...   
1  In accordance with an example embodiment, a so...   
2  Verfahren zum Steuern einer Windenergieanlage ...   
3  Verfahren zum Steuern einer Windenergieanlage ...   
4  A control method for optimizing a solar-to-pow...   

                                          title_text  
0  Adaptable DC-AC Inverter Drive System and Oper...  
1  System for providing the energy from a single ...  
2      Verfahren zum steuern einer windenergieanlage  
3      Verfahren zum steuern einer windenergieanlage  
4  Control method for optimizing solar-to-power e...  


In [178]:
import re

def clean_text(text):
    if pd.isna(text):
        return ""
    # Convert to lower case
    text = text.lower()
    # Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    # Remove unwanted characters, but preserve hyphens within words
    text = re.sub(r'[^a-z\s-]', '', text)
    # Clean up any stray hyphens that do not form part of a hyphenated word
    text = re.sub(r'\b-\b', ' ', text)
    # Remove multiple spaces and trim
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Apply cleaning function to both columns
results_json['abstract_text'] = results_json['abstract_text'].apply(clean_text)
results_json['title_text'] = results_json['title_text'].apply(clean_text)

# Print cleaned text for inspection
print(results_json[['abstract_text', 'title_text']].head())

                                       abstract_text  \
0  disclosed is an adaptable dc ac inverter syste...   
1  in accordance with an example embodiment a sol...   
2  verfahren zum steuern einer windenergieanlage ...   
3  verfahren zum steuern einer windenergieanlage ...   
4  a control method for optimizing a solar to pow...   

                                          title_text  
0  adaptable dc ac inverter drive system and oper...  
1  system for providing the energy from a single ...  
2      verfahren zum steuern einer windenergieanlage  
3      verfahren zum steuern einer windenergieanlage  
4  control method for optimizing solar to power e...  


In [179]:
import re
from collections import Counter

# Function to list unique characters in the text
def find_unique_characters(text_series):
    unique_chars = set()
    for text in text_series:
        unique_chars.update(text)
    return unique_chars

# Function to count tokens in the text
def count_tokens(text_series):
    token_counts = []
    for text in text_series:
        # Simple tokenization based on spaces
        tokens = text.split()
        token_counts.append(len(tokens))
    return token_counts

# Apply functions to your data
unique_chars_abstracts = find_unique_characters(results_json['abstract_text'])
unique_chars_titles = find_unique_characters(results_json['title_text'])
abstract_token_counts = count_tokens(results_json['abstract_text'])
title_token_counts = count_tokens(results_json['title_text'])

# Print results for inspection
print("Unique characters in abstracts:", unique_chars_abstracts)
print("Unique characters in titles:", unique_chars_titles)
print("Token counts in abstracts:", abstract_token_counts)
print("Token counts in titles:", title_token_counts)
## too many zero tokens!! 

Unique characters in abstracts: {'q', 'e', 'h', 'k', 'u', 'v', 'o', 'a', 'x', 'b', 'n', 'l', 's', 'g', 'c', 'y', 'd', 'w', 'f', ' ', 'z', 'j', 'r', 'i', 'p', '-', 't', 'm'}
Unique characters in titles: {'q', 'e', 'h', 'k', 'u', 'v', 'o', 'a', 'x', 'n', 'b', 's', 'l', 'g', 'c', 'y', 'd', 'w', 'f', ' ', 'z', 'j', 'r', 'i', 'p', '-', 't', 'm'}
Token counts in abstracts: [67, 92, 208, 208, 166, 0, 1, 72, 1, 0, 1, 1, 210, 163, 0, 0, 1, 227, 126, 0, 0, 0, 188, 0, 0, 210, 0, 0, 179, 0, 204, 0, 0, 205, 0, 0, 246, 0, 255, 150, 0, 186, 0, 213, 0, 0, 1, 0, 0, 180, 0, 169, 146, 0, 282, 0, 0, 1, 0, 0, 0, 0, 200, 0, 168, 219, 0, 0, 217, 134, 0, 0, 193, 0, 223, 202, 114, 0, 0, 0, 0, 1, 187, 180, 0, 0, 217, 169, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 282, 1, 0, 179, 0, 0, 0, 0, 0, 0, 282, 0, 0, 0, 235, 186, 1, 210, 202, 0, 146, 217, 1, 231, 1, 217, 0, 181, 0, 0, 0, 0, 0, 0, 0, 250, 0, 186, 200, 180, 212, 0, 231, 222, 0, 1, 0, 200, 227, 205, 0, 0, 0, 186, 168, 0, 5, 213, 150, 121, 0, 0, 266, 255, 0, 97, 0

In [180]:
# Example to check entries with zero tokens
zero_token_indices = [i for i, count in enumerate(abstract_token_counts) if count == 0]
print("Entries with zero tokens:")
for index in zero_token_indices:
    print(index, results_json['abstract_text'].iloc[index])

# Example to check entries with very high token counts
high_token_indices = [i for i, count in enumerate(abstract_token_counts) if count > 200]  # arbitrary threshold
print("\nEntries with high tokens:")
for index in high_token_indices:
    print(index, results_json['abstract_text'].iloc[index])


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [181]:
import numpy as np
## SAMPLING THE DATA
# Sample up to 5 entries with zero tokens
sampled_zero_token_indices = np.random.choice(zero_token_indices, size=min(5, len(zero_token_indices)), replace=False)
print("Sampled entries with zero tokens:")
for index in sampled_zero_token_indices:
    print(index, results_json['abstract_text'].iloc[index])

# Sample up to 5 entries with very high token counts
sampled_high_token_indices = np.random.choice(high_token_indices, size=min(5, len(high_token_indices)), replace=False)
print("\nSampled entries with high tokens:")
for index in sampled_high_token_indices:
    print(index, results_json['abstract_text'].iloc[index])


Sampled entries with zero tokens:
10033 
2628 
27993 
17855 
17241 

Sampled entries with high tokens:
29664 the present disclosure pertains to the technical field of photovoltaics and provides a back contact solar cell assembly the assembly comprises at least one cell string comprising multiple back contact solar cells and multiple conductive wires first electrodes each of the first electrodes comprising a first busbar and a first finger and second electrodes each of the second electrodes comprising a second busbar and a second finger the conductive wires comprise a first portion and a second portion the first portion is used to electrically connect the first busbar of a first back contact solar cell and the second portion is used to electrically connect the second busbar of an adjacent second back contact solar cell the first portion is provided with at least one deformation buffer structure on the non light receiving surface of the first back contact solar cell and in the direction 

In [182]:
#WRITING TO A FILE TO INSPECT

# Writing entries with zero tokens to a file
with open('zero_tokens_entries.txt', 'w') as file:
    for index in zero_token_indices:
        file.write(f"{index}: {results_json['abstract_text'].iloc[index]}\n")

# Writing entries with very high token counts to a file
with open('high_tokens_entries.txt', 'w') as file:
    for index in high_token_indices:
        file.write(f"{index}: {results_json['abstract_text'].iloc[index]}\n")

print("Entries have been written to files: zero_tokens_entries.txt and high_tokens_entries.txt")


Entries have been written to files: zero_tokens_entries.txt and high_tokens_entries.txt


#### Handle duplicates - don't run yet 


We decided to handle duplicates after convertion of columns (for easier process and to avoid overwriting the code)

In [ ]:
# Display the first few rows to ensure duplicates have been removed
print(results_json.head())

# Check for any remaining duplicates
print(f"Remaining duplicates: {results_json.duplicated().sum()}")


In [ ]:
#Drop duplicates JSON
# Convert list columns to strings
for col in results_json.columns:
    if isinstance(results_json[col].iloc[0], list):  # Check if the first cell contains a list
        results_json[col] = results_json[col].apply(lambda x: json.dumps(x))


In [ ]:
# Ensure all entries in 'inventor' are stringified JSON
results_json['inventor'] = results_json['inventor'].apply(
    lambda x: json.dumps(x) if isinstance(x, list) else '[]' if x is None else x
)

# Verify the conversion once more
print("After conversion:")
for col in results_json.columns:
    print(f"{col} type:", type(results_json[col].iloc[0]))

# Now try to find duplicates again
duplicates = results_json.duplicated()
print(f"There are {duplicates.sum()} duplicate rows.")


In [ ]:

# Now you can drop duplicates
#results_json.drop_duplicates(inplace=True)

## CSV PROCESSING

#### CT_Evaluation cleaning

In [186]:
#Look into data frame already uploaded at the beginning of the notebook

print("First few rows:")
print(ct_evaluation.head())


First few rows:
   example_id  question_id                                           question  \
0           1            1  What is the innovation behind Leclanché's new ...   
1           2            2       What is the EU’s Green Deal Industrial Plan?   
2           3            2       What is the EU’s Green Deal Industrial Plan?   
3           4            3  What are the four focus areas of the EU's Gree...   
4           5            4  When did the cooperation between GM and Honda ...   

                                      relevant_chunk  \
0  Leclanché said it has developed an environment...   
1  The Green Deal Industrial Plan is a bid by the...   
2  The European counterpart to the US Inflation R...   
3  The new plan is fundamentally focused on four ...   
4  What caught our eye was a new hookup between G...   

                                         article_url  
0  https://www.sgvoice.net/strategy/technology/23...  
1  https://www.sgvoice.net/policy/25396/eu-seeks-.

In [188]:
# Check for missing values in each column
missing_data = ct_evaluation.isnull().sum()
print("Missing values in each column:")
print(missing_data)
# No missing data, which simplyify our cleaning steps

Missing values in each column:
example_id        0
question_id       0
question          0
relevant_chunk    0
article_url       0
dtype: int64


In [196]:
import pandas as pd

def generate_summary(df):
    # Identify non-hashable columns (e.g., columns with lists)
    non_hashable_columns = [col for col in df.columns if isinstance(df[col].dropna().iloc[0], list)]
    
    # Exclude non-hashable columns from the nunique calculation
    hashable_columns = df.drop(columns=non_hashable_columns)
    
    # Essentials: type, unique values, missing values
    summary = pd.DataFrame({
        'type': df.dtypes,
        'unique_values': hashable_columns.nunique(),
        'missing_values': df.isnull().sum()
    })
    
    # Quantile statistics for numeric columns only
    numeric_cols = df.select_dtypes(include='number')
    quantiles = numeric_cols.quantile([0, 0.25, 0.5, 0.75, 1]).transpose()
    
    # Add quantile stats to the summary
    summary_stats = pd.concat([summary, quantiles], axis=1, sort=False)
    
    # Compute range and interquartile range (IQR)
    if not numeric_cols.empty:  # Only calculate if there are numeric columns
        summary_stats['range'] = summary_stats[1.0] - summary_stats[0.0]
        summary_stats['iqr'] = summary_stats[0.75] - summary_stats[0.25]
    
    return summary_stats

# Apply the summary function to your datasets
ct_evaluation_summary = generate_summary(ct_evaluation)
# ct_media_summary = generate_summary(ct_media) # Uncomment and replace with your actual DataFrame name

# Display the summaries
print("Summary for ct_evaluation:")
print(ct_evaluation_summary)


Summary for ct_evaluation:
                         type  unique_values  missing_values   0.0  0.25  \
article_url            object           21.0               0   NaN   NaN   
chunk_length            int64           22.0               0  17.0  28.5   
example_id              int64           23.0               0   1.0   6.5   
question               object           21.0               0   NaN   NaN   
question_id             int64           21.0               0   1.0   5.5   
question_length         int64           12.0               0   4.0   9.0   
question_tokens        object            NaN               0   NaN   NaN   
relevant_chunk         object           23.0               0   NaN   NaN   
relevant_chunk_tokens  object            NaN               0   NaN   NaN   

                        0.5  0.75    1.0  range   iqr  
article_url             NaN   NaN    NaN    NaN   NaN  
chunk_length           40.0  61.5  109.0   92.0  33.0  
example_id             12.0  17.5   23.0   2

In [189]:
# Look into unique characters to indentify which ones to remove
# Create sets to hold unique characters
unique_chars_question = set()
unique_chars_chunk = set()

# Update sets with characters from each row
for question, chunk in zip(ct_evaluation['question'], ct_evaluation['relevant_chunk']):
    unique_chars_question.update(question)
    unique_chars_chunk.update(chunk)

# Print the unique characters
print("Unique characters in 'question' column:")
print(sorted(unique_chars_question))
print("\nUnique characters in 'relevant_chunk' column:")
print(sorted(unique_chars_chunk))

Unique characters in 'question' column:
[' ', '"', "'", ',', '-', '0', '1', '2', '3', '5', '?', 'A', 'C', 'D', 'E', 'G', 'H', 'I', 'L', 'M', 'P', 'S', 'U', 'W', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'y', 'é', '’']

Unique characters in 'relevant_chunk' column:
[' ', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '8', '9', ':', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'é', '–', '’', '“', '”']


In [190]:
import unicodedata

def standardize_text(text):
    # Normalize and convert to lowercase
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8').lower()
    
    # Replace special apostrophes and quotation marks
    text = text.replace('’', "'").replace('“', '"').replace('”', '"')
    
    # Standardize dashes
    text = text.replace('–', '-')  # Replace en-dash with hyphen
    
    return text

# Apply the cleaning function to both 'question' and 'relevant_chunk' columns
ct_evaluation['question'] = ct_evaluation['question'].apply(standardize_text)
ct_evaluation['relevant_chunk'] = ct_evaluation['relevant_chunk'].apply(standardize_text)

# Display the cleaned text to confirm changes
print("Standardized 'question' column:")
print(ct_evaluation['question'].head())
print("\nStandardized 'relevant_chunk' column:")
print(ct_evaluation['relevant_chunk'].head())

Standardized 'question' column:
0    what is the innovation behind leclanche's new ...
1          what is the eus green deal industrial plan?
2          what is the eus green deal industrial plan?
3    what are the four focus areas of the eu's gree...
4    when did the cooperation between gm and honda ...
Name: question, dtype: object

Standardized 'relevant_chunk' column:
0    leclanche said it has developed an environment...
1    the green deal industrial plan is a bid by the...
2    the european counterpart to the us inflation r...
3    the new plan is fundamentally focused on four ...
4    what caught our eye was a new hookup between g...
Name: relevant_chunk, dtype: object


In [192]:
import re
from nltk.tokenize import word_tokenize
import nltk

# Ensure you've downloaded the necessary NLTK resource
nltk.download('punkt')

# Define the tokenization function using NLTK
def tokenize_text_nltk(text):
    tokens = word_tokenize(text)
    return tokens, len(tokens)

# Apply the tokenization function to the 'question' and 'relevant_chunk' columns
ct_evaluation['question_tokens'], ct_evaluation['question_length'] = zip(*ct_evaluation['question'].apply(tokenize_text_nltk))
ct_evaluation['relevant_chunk_tokens'], ct_evaluation['chunk_length'] = zip(*ct_evaluation['relevant_chunk'].apply(tokenize_text_nltk))

# Display the first few tokenized entries and their lengths to confirm
print("Tokenized 'question' column with lengths:")
for tokens, length in zip(ct_evaluation['question_tokens'].head(), ct_evaluation['question_length'].head()):
    print(f"{length} tokens: {tokens}")

print("\nTokenized 'relevant_chunk' column with lengths:")
for tokens, length in zip(ct_evaluation['relevant_chunk_tokens'].head(), ct_evaluation['chunk_length'].head()):
    print(f"{length} tokens: {tokens}")


Tokenized 'question' column with lengths:
14 tokens: ['what', 'is', 'the', 'innovation', 'behind', 'leclanche', "'s", 'new', 'method', 'to', 'produce', 'lithium-ion', 'batteries', '?']
9 tokens: ['what', 'is', 'the', 'eus', 'green', 'deal', 'industrial', 'plan', '?']
9 tokens: ['what', 'is', 'the', 'eus', 'green', 'deal', 'industrial', 'plan', '?']
15 tokens: ['what', 'are', 'the', 'four', 'focus', 'areas', 'of', 'the', 'eu', "'s", 'green', 'deal', 'industrial', 'plan', '?']
14 tokens: ['when', 'did', 'the', 'cooperation', 'between', 'gm', 'and', 'honda', 'on', 'fuel', 'cell', 'vehicles', 'start', '?']

Tokenized 'relevant_chunk' column with lengths:
44 tokens: ['leclanche', 'said', 'it', 'has', 'developed', 'an', 'environmentally', 'friendly', 'way', 'to', 'produce', 'lithium-ion', '(', 'li-ion', ')', 'batteries', '.', 'it', 'has', 'replaced', 'highly', 'toxic', 'organic', 'solvents', ',', 'commonly', 'used', 'in', 'the', 'production', 'process', ',', 'with', 'a', 'water-based', 'proc

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\maaru\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [193]:
### URL cleaning

# Summary of the article_url column
url_summary = ct_evaluation['article_url'].agg(['dtype', 'nunique', 'count', pd.Series.nunique, pd.Series.unique])

# Check for missing values
url_summary['missing_values'] = ct_evaluation['article_url'].isnull().sum()

# Display the summary
print(url_summary)


dtype                                                        object
nunique                                                          21
count                                                            23
nunique                                                          21
unique            [https://www.sgvoice.net/strategy/technology/2...
missing_values                                                    0
Name: article_url, dtype: object


In [197]:
import pandas as pd
from urllib.parse import urlparse, urlunparse

def clean_url(url):
    # Parse the URL
    parsed_url = urlparse(url.lower())
    # Reconstruct without query and fragment
    cleaned_url = urlunparse((parsed_url.scheme, parsed_url.netloc, parsed_url.path, '', '', ''))
    return cleaned_url

def extract_domain(url):
    # Parse the URL and extract the domain
    domain = urlparse(url.lower()).netloc
    return domain

# Apply cleaning to the article_url column
ct_evaluation['cleaned_article_url'] = ct_evaluation['article_url'].apply(clean_url)
ct_evaluation['domain'] = ct_evaluation['article_url'].apply(extract_domain)

# Display the cleaned URLs and domains
print(ct_evaluation[['article_url', 'cleaned_article_url', 'domain']])

                                          article_url  \
0   https://www.sgvoice.net/strategy/technology/23...   
1   https://www.sgvoice.net/policy/25396/eu-seeks-...   
2   https://www.pv-magazine.com/2023/02/02/europea...   
3   https://www.sgvoice.net/policy/25396/eu-seeks-...   
4   https://cleantechnica.com/2023/05/08/general-m...   
5   https://solarindustrymag.com/scout-and-colgate...   
6   https://cleantechnica.com/2023/01/02/the-wait-...   
7   https://cleantechnica.com/2023/05/15/what-does...   
8   https://cleantechnica.com/2023/05/15/mississip...   
9   https://cleantechnica.com/2023/05/18/solar-pan...   
10  https://www.azocleantech.com/news.aspx?newsID=...   
11  https://www.azocleantech.com/news.aspx?newsID=...   
12  https://www.azocleantech.com/news.aspx?newsID=...   
13  https://www.azocleantech.com/news.aspx?newsID=...   
14  https://www.azocleantech.com/news.aspx?newsID=...   
15  https://www.thinkgeoenergy.com/japan-and-icela...   
16  https://www.thinkgeoenergy.

#### CT_Media Cleaning

In [187]:
print("First few rows:")
print(ct_media.head())


First few rows:
   Unnamed: 0                                              title        date  \
0        1280  Qatar to Slash Emissions as LNG Expansion Adva...  2021-01-13   
1        1281               India Launches Its First 700 MW PHWR  2021-01-15   
2        1283              New Chapter for US-China Energy Trade  2021-01-20   
3        1284  Japan: Slow Restarts Cast Doubt on 2030 Energy...  2021-01-22   
4        1285     NYC Pension Funds to Divest Fossil Fuel Shares  2021-01-25   

  author                                            content       domain  \
0    NaN  ["Qatar Petroleum ( QP) is targeting aggressiv...  energyintel   
1    NaN  ["• Nuclear Power Corp. of India Ltd. ( NPCIL)...  energyintel   
2    NaN  ["New US President Joe Biden took office this ...  energyintel   
3    NaN  ["The slow pace of Japanese reactor restarts c...  energyintel   
4    NaN  ["Two of New York City's largest pension funds...  energyintel   

                                              

In [194]:
import pandas as pd

def generate_summary(df):
    # Essentials: type, unique values, missing values
    summary = pd.DataFrame({
        'type': df.dtypes,
        'unique_values': df.nunique(),
        'missing_values': df.isnull().sum()
    })
    
    # Quantile statistics for numeric columns only
    numeric_cols = df.select_dtypes(include='number')
    quantiles = numeric_cols.quantile([0, 0.25, 0.5, 0.75, 1]).transpose()
    
    # Add quantile stats to the summary
    summary_stats = pd.concat([summary, quantiles], axis=1)
    
    # Compute range and interquartile range (IQR)
    summary_stats['range'] = summary_stats[1.0] - summary_stats[0.0]
    summary_stats['iqr'] = summary_stats[0.75] - summary_stats[0.25]
    
    return summary_stats

# Apply the summary function to both datasets
ct_media_summary = generate_summary(ct_media)

# Display the summaries
print("\nSummary for ct_media:")
print(ct_media_summary)


Summary for ct_media:
              type  unique_values  missing_values     0.0     0.25      0.5  \
Unnamed: 0   int64           9593               0  1280.0  17156.0  45991.0   
title       object           9569               0     NaN      NaN      NaN   
date        object            967               0     NaN      NaN      NaN   
author      object              7            9562     NaN      NaN      NaN   
content     object           9588               0     NaN      NaN      NaN   
domain      object             19               0     NaN      NaN      NaN   
url         object           9593               0     NaN      NaN      NaN   

               0.75      1.0    range      iqr  
Unnamed: 0  79250.0  81816.0  80536.0  62094.0  
title           NaN      NaN      NaN      NaN  
date            NaN      NaN      NaN      NaN  
author          NaN      NaN      NaN      NaN  
content         NaN      NaN      NaN      NaN  
domain          NaN      NaN      NaN      NaN  
u

In [199]:
# Handle date, look into consistency and earlies and latest date
# Check the earliest and latest dates
print("Earliest date:", ct_media['date'].min())
print("Latest date:", ct_media['date'].max())

# Check the frequency of each date to identify any unusual patterns
date_counts = ct_media['date'].value_counts().sort_index()
print("Date frequencies:\n", date_counts.head())

Earliest date: 2021-01-01 00:00:00
Latest date: 2023-12-05 00:00:00
Date frequencies:
 2021-01-01    3
2021-01-02    1
2021-01-03    2
2021-01-04    5
2021-01-05    8
Name: date, dtype: int64


In [203]:
import pandas as pd

def unique_characters(column):
    return set(''.join(column))

# Load your data (assuming DataFrame is named ct_media)
# ct_media = pd.read_csv('path_to_your_file.csv')

# Inspect unique characters in the title
title_chars = unique_characters(ct_media['title'])
print("Unique characters in 'title':", title_chars)

# Inspect unique characters in the content
content_chars = unique_characters(ct_media['content'])
print("Unique characters in 'content':", content_chars)


Unique characters in 'title': {'ü', 'ç', 'L', ')', 'U', 'í', '9', 'ä', 'u', 'ú', 'C', 'a', 'N', '™', 'P', 'b', '₂', 's', 'l', '⅓', '@', 'y', '&', '%', 'R', '‘', 'Z', '5', '×', '!', '+', 'J', 'H', "'", 'á', 'W', 'q', '°', '1', 'o', 'x', '[', '?', '6', 'g', 'Ø', 'É', ';', 'ö', '=', 'D', '/', ']', 'f', ' ', 'B', 'p', 'm', 'M', 'ë', 'E', 'e', 'k', 'T', '½', '2', '€', '’', 'ø', 'Q', '“', '–', '>', 'I', 'F', 'â', '\u200b', '0', '3', 'w', 'é', '4', 'j', '(', '£', 'i', '…', '-', '.', '®', 'à', '7', '”', '⅙', '#', '`', 'h', ':', 'Ö', 'X', 'v', 'A', 'V', ',', '*', 'n', 'c', 'å', 'K', 'ó', '—', 'O', 'd', 'ē', '$', '~', 'S', 'z', 'Y', 'r', 'G', '8', 't'}
Unique characters in 'content': {'ç', 'ï', '开', '🎉', 'ż', '9', 'ª', 'C', 'வ', 'æ', 'a', 'Ü', 'l', '⋅', 'Ł', 'ū', 'Í', '😳', 'м', '‑', 'ு', 'Ç', '🛢', 'Z', '🎄', 'ê', '×', 'ɑ', 'ù', 'H', 'χ', '}', 'Т', '‰', 'W', 'q', 'О', '🚀', 'р', '👏', '😍', '©', '，', 'x', '[', '6', 'й', '😀', 'Ø', 'с', '·', ']', '회', 'Æ', 'д', '👇', 'M', '"', '毛', 'ы', '℃', 'k', '🇸', '

### Cleaning Title

In [206]:
import unicodedata
import re

def clean_title(text):
    # Normalize unicode characters and convert to lower
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8').lower()
    # Remove special characters
    text = re.sub(r'[^\w\s-]', '', text)
    return text

# Apply the cleaning function to the title column
ct_media['title'] = ct_media['title'].apply(clean_title)

# Display the cleaned titles
print(ct_media['title'].head())

#examine unique characters again
title_chars = unique_characters(ct_media['title'])
print("Unique characters in 'title':", title_chars)

0    qatar to slash emissions as lng expansion adva...
1                 india launches its first 700 mw phwr
2                new chapter for us-china energy trade
3    japan slow restarts cast doubt on 2030 energy ...
4       nyc pension funds to divest fossil fuel shares
Name: title, dtype: object
Unique characters in 'title': {'7', 'q', 'e', 'h', 'k', '9', 'u', '1', 'v', '2', 'o', 'a', 'x', '6', 'n', 'b', 'l', 's', 'g', 'c', 'y', '0', '3', 'd', 'w', '5', '4', 'f', ' ', 'j', 'z', 'r', 'i', 'p', '8', '-', 't', 'm'}


### Cleaning content

In [208]:
def clean_content(text):
    # Normalize unicode characters
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8').lower()
    # Remove HTML tags (if any)
    text = re.sub(r'<[^>]+>', '', text)
    # Remove emojis
    text = re.sub(r'[^\w\s.,-]', '', text)
    return text

# Apply the cleaning function to the content column
ct_media['content'] = ct_media['content'].apply(clean_content)

# Display the cleaned content
print(ct_media['content'].head())

# Inspect unique characters in the content
content_chars = unique_characters(ct_media['content'])
print("Unique characters in 'content':", content_chars)

0    qatar petroleum  qp is targeting aggressive cu...
1     nuclear power corp. of india ltd.  npcil sync...
2    new us president joe biden took office this we...
3    the slow pace of japanese reactor restarts con...
4    two of new york citys largest pension funds sa...
Name: content, dtype: object
Unique characters in 'content': {'7', 'q', 'e', 'h', 'k', '9', 'u', '1', 'v', '2', 'o', 'a', ',', 'x', '6', 'n', 'b', 's', 'l', 'g', 'c', 'y', '0', '3', 'd', 'w', '5', '4', 'f', ' ', 'j', 'z', 'r', 'i', 'p', '8', '-', 't', 'm', '.'}


#### TOKANIZATION


In [210]:
# Tokenization of the title
ct_media['tokenized_title'] = ct_media['title'].apply(word_tokenize)

# Count tokens in both content and title
ct_media['content_token_count'] = ct_media['tokenized_content'].apply(len)
ct_media['title_token_count'] = ct_media['tokenized_title'].apply(len)

# Display the first few entries with token counts
print(ct_media[['tokenized_title', 'title_token_count', 'tokenized_content', 'content_token_count']].head())


                                     tokenized_title  title_token_count  \
0  [qatar, to, slash, emissions, as, lng, expansi...                  8   
1       [india, launches, its, first, 700, mw, phwr]                  7   
2       [new, chapter, for, us-china, energy, trade]                  6   
3  [japan, slow, restarts, cast, doubt, on, 2030,...                  9   
4  [nyc, pension, funds, to, divest, fossil, fuel...                  8   

                                   tokenized_content  content_token_count  
0  [qatar, petroleum, qp, is, targeting, aggressi...                  472  
1  [nuclear, power, corp., of, india, ltd., npcil...                  579  
2  [new, us, president, joe, biden, took, office,...                  767  
3  [the, slow, pace, of, japanese, reactor, resta...                  737  
4  [two, of, new, york, citys, largest, pension, ...                  421  


#### Handle missing values in author - split at the end to create two separate dataframes


In [200]:
# Creating a DataFrame only with entries that have an author
ct_media_with_authors = ct_media.dropna(subset=['author'])

# Display the new DataFrame
print(ct_media_with_authors.head())

# Creating a DataFrame for entries without an author
ct_media_without_authors = ct_media[ct_media['author'].isnull()]

# Display the DataFrame without authors
print(ct_media_without_authors.head())


      Unnamed: 0                                              title  \
5564       76234  Ecotricity strikes landmark deal to secure geo...   
5565       76235  'Getting to Zero ': New commission to explore ...   
5566       76236  From floating turbines to carbon capture x-ray...   
5567       76237  Electrification of harbour crane moves Port of...   
5568       76238                            No hugging, no learning   

           date               author  \
5564 2021-01-05      Cecilia Keating   
5565 2021-01-05       James S Murray   
5566 2021-01-06  BusinessGreen staff   
5567 2021-01-06      Cecilia Keating   
5568 2021-01-06       James S Murray   

                                                content         domain  \
5564  ["Geothermal energy in the UK took a major ste...  businessgreen   
5565  ['Up to 10 million jobs across the UK are in c...  businessgreen   
5566  ['UK science laboratories and testing faciliti...  businessgreen   
5567  ['The Port of Tyne has made ma

In [202]:
#Inspect the separate dataset

# Check the size of both datasets
print("Entries with authors:", ct_media_with_authors.shape[0])
print("Entries without authors:", ct_media_without_authors[ct_media_without_authors['author'].isnull()].shape[0])

# Inspect the first few entries of the dataset with authors
print("\nFirst few entries with authors:")
print(ct_media_with_authors.head())

# Inspect the first few entries of the dataset without authors
print("\nFirst few entries without authors:")
print(ct_media_without_authors[ct_media_without_authors['author'].isnull()].head())

# Analyze domains to see if there are patterns related to missing authors
print("\nDomain counts for entries with authors:")
print(ct_media_with_authors['domain'].value_counts())

print("\nDomain counts for entries without authors:")
print(ct_media_without_authors[ct_media_without_authors['author'].isnull()]['domain'].value_counts())


Entries with authors: 31
Entries without authors: 9562

First few entries with authors:
      Unnamed: 0                                              title  \
5564       76234  Ecotricity strikes landmark deal to secure geo...   
5565       76235  'Getting to Zero ': New commission to explore ...   
5566       76236  From floating turbines to carbon capture x-ray...   
5567       76237  Electrification of harbour crane moves Port of...   
5568       76238                            No hugging, no learning   

           date               author  \
5564 2021-01-05      Cecilia Keating   
5565 2021-01-05       James S Murray   
5566 2021-01-06  BusinessGreen staff   
5567 2021-01-06      Cecilia Keating   
5568 2021-01-06       James S Murray   

                                                content         domain  \
5564  ["Geothermal energy in the UK took a major ste...  businessgreen   
5565  ['Up to 10 million jobs across the UK are in c...  businessgreen   
5566  ['UK science lab